2D Sampling
========
-----------------------------

In [ ]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import scipy.misc

--------------------
Define a function that subsamples the image


In [ ]:
def subsample (img,n):
    [lines,columns]= img.shape
    img_sub_sampled = img[0:lines:n, 0:columns:n]
    return img_sub_sampled

--------------------
Load and subsample the image

In [ ]:
path_image = path.join('data','circles.png')
img = plt.imread(path_image,0)

subsampling_factor = 4

img_subsampled = subsample(img,subsampling_factor)

path_image_subsampled = path.join('data','circles_subsampled.png')
plt.imsave(path_image_subsampled,img_subsampled, cmap='gray')
print(img.shape)
print(img_subsampled.shape)

In [ ]:
plt.figure(1, figsize = (10,4))

plt.subplot(121)
plt.imshow(img, cmap = 'gray')
plt.title("Circles")

plt.subplot(122)
plt.imshow(img_subsampled, cmap = 'gray')
plt.title("Subsampled circles")

plt.show()

--------------------
Show the **spectrum** of the image before and after the subsampling.


In [ ]:
def spectrum(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)
    return magnitude_spectrum

In [ ]:
spectrum_original = spectrum(img)
spectrum_subsampled = spectrum(img_subsampled)

plt.figure(1, figsize = (10,4))

plt.subplot(121)
plt.imshow(np.log(spectrum_original), cmap = 'gray')
plt.title('Spectrum original img')

plt.subplot(122)
plt.imshow(np.log(spectrum_subsampled), cmap = 'gray')
plt.title('Spectrum subsampled img')
plt.show()

Filter the image in frequency space
===


In [ ]:
rows, cols = img.shape

# create square mask with ones in the center and zero everywhere else
square = np.zeros((rows,cols),np.uint8)
N = 64
square[(rows-N)/2:(rows+N)/2, (cols-N)/2:(cols+N)/2] = 1

plt.figure()
plt.imshow(square, cmap = 'gray')
plt.title('Template square filter')
plt.show()

# Image in frequency space
img_freq = np.fft.fft2(img) # F(u,v)

# Shift the square filter
filt_freq_shift = np.fft.fftshift(square) 

# Filter in frequency space
img_filt = img_freq*filt_freq_shift # G(u,v) = F(u,v)*H(u,v)

# Just to understand why fftshift is necessary
plt.figure(7,figsize = (20,5))

plt.subplot(131)
plt.plot(img_freq)
plt.title('Image in frequency space')

plt.subplot(132)
plt.plot(square)
plt.title('Filter')

plt.subplot(133)
plt.plot(filt_freq_shift)
plt.title('Shifted filter')

plt.show()

Compare the spectrum before and after the frequency filtering
====

In [ ]:
spectrum_original = spectrum(img)
spectrum_filtered = np.log(np.abs(np.fft.fftshift(img_filt))) 

plt.figure(8,figsize = (15,5))
plt.subplot(121)
plt.imshow(np.log(spectrum_original), cmap = 'gray')
plt.title('Spectrum original image')
plt.subplot(122)
plt.imshow(spectrum_filtered, cmap = 'gray')
plt.title('Spectrum filtered image')
plt.show()

Back to real space
====

In [ ]:
img_real =  np.abs(np.fft.ifft2(img_filt))

plt.figure(figsize = (10,5))
plt.imshow(img_real,cmap = 'gray')
plt.show()

In [ ]:
img_filt_subsampled = subsample(img_real, 4)

plt.figure(figsize = (10,5))
plt.imshow(img_filt_subsampled, cmap = 'gray')
plt.show()

-------------------
Ideal lowpass filter
====
An ideal lowpass filter keeps all the frequences (without attenuation) within a circle centered in the origin and cancels all the others.

In [ ]:
M, N = img.shape

x,y = np.mgrid[-M/2:M/2:1,-N/2:N/2:1]

disk = (x**2+y**2 <= (N/8)**2).astype(float)
disk_freq = np.fft.fftshift(disk)

img_filtered_disk = img_freq*disk_freq

spectrum_filt = np.log(np.abs(np.fft.fftshift(img_filtered_disk)+1)) 

# Plot disk and spectrum
plt.figure(7,figsize = (20,5))

plt.subplot(131)
plt.imshow(disk, cmap = 'gray')
plt.title('Disk')

plt.subplot(132)
plt.imshow(np.log(spectrum_original), cmap = 'gray')
plt.title('Spectrum')

plt.subplot(133)
plt.imshow(spectrum_filt, cmap = 'gray')
plt.title('Product in frequency space')
plt.show()

In [ ]:
# Why do we need the fftshift?
plt.figure(7,figsize = (20,5))

plt.subplot(131)
plt.plot(img_freq)
plt.title('Image en frequence')

plt.subplot(132)
plt.plot(disk)
plt.title('disque')

plt.subplot(133)
plt.plot(disk_freq)
plt.title('ffshift filtre disque')

plt.show()

In [ ]:
img_space_disk =  np.abs(np.fft.ifft2(img_filtered_disk))

plt.figure(figsize = (10,5))

plt.subplot(131)
plt.imshow(img_space_disk, cmap = 'gray')
plt.title('low pass filtered')

plt.subplot(132)
plt.imshow(img, cmap = 'gray')
plt.title('original')

plt.subplot(133)
plt.imshow(img_subsampled, cmap = 'gray')
plt.title('subsampled')

plt.show()


In [ ]:
#Image filtree decimee
img_subsampled_filtered_disk = subsample(img_space_disk, subsampling_factor)

plt.figure(figsize = (10,5))
plt.imshow(img_subsampled_filtered_disk,cmap = 'gray')
plt.title('Subsampled disk-filtered image')
plt.show()


Gaussian filter
===

Documentation of the function: [https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html)

In [ ]:
from scipy.ndimage.filters import gaussian_filter
img_filtered_gaussian = gaussian_filter(img, sigma = 1)

In [ ]:
plt.figure(1, figsize = (15,8))

plt.subplot(131)
plt.imshow(img, cmap = 'gray')
plt.title("Circles")

plt.subplot(132)
plt.imshow(img_subsampled, cmap = 'gray')
plt.title("Subsampled circles")

plt.subplot(133)
plt.imshow(img_filtered_gaussian,cmap = 'gray')
plt.title('Image after gaussian filtering')

plt.show()

plt.imsave(path.join('data','gaussian_filtering.png'), img_filtered_gaussian, cmap = 'gray')